# Clinical Decision Question Answer System 关键点分析

## 1.Action代码分析
在分析之前，我们将action代码部分分为几个模块，按照模块来分析
* 链接图数据库部分
```
graph = Graph(host="119.45.121.125", http_port=7474, user="neo4j", password="123456"
```
我们在安装好图数据库之后，只需要将对应得host，port，用户名密码填入即可链接，如果需要执行某个查询：
```
graph.run("match (a:Disease{name: {disease}}) return a", disease=disease).data()
```
* 同名疾病的查询
```
p = 'data/medical/lookup/Diseases.txt' #读取所有的疾病
disease_names = [i.strip() for i in open(p, 'r', encoding='UTF-8').readlines()]#把所有的疾病放到内存变量里
def retrieve_disease_name(name):
    names = []
    name = '.*' + '.*'.join(list(name)) + '.*' #构造模糊查询，比如输入肺炎，可以把各种肺炎返回，生成细粒度的肺炎病种供用户选择
    pattern = re.compile(name)
    for i in disease_names:
        candidate = pattern.search(i)
        if candidate:
            names.append(candidate.group())
    return names
```

* 初始化action以及fallbackaction
初始化action可以理解为冷启动机器人的回复内容，我们在打开聊天界面，希望机器人主动问好，主动告诉我们它是什么一个机器人，以及怎么用这个机器人，因此我们只需要定义这样一个action，在story里将这个action执行的顺序排第一，那么机器人即可知道只要触发会话就应该执行这个二action，在action里我们再去定义几个模板，即可达到我们的目的：
```
class ActionFirst(Action):
    def name(self) -> Text:
        return "action_first" #action的名字，是必不可少的，在domain文件以及story文件怎么引用action，都是靠这个名字来索引的
    def run(self,
            dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]):
        dispatcher.utter_template("utter_first", tracker)#代表会返回在domain文件里定义的utter_first内容
        dispatcher.utter_message(md("您可以这样向我提问: "
                                    "<br/>头痛怎么办<br/>\
                              什么人容易头痛<br/>\
                              头痛吃什么药<br/>\
                              头痛能治吗<br/>\
                              头痛属于什么科<br/>\
                              头孢地尼分散片用途<br/>\
                              如何防止头痛<br/>\
                              头痛要治多久<br/>\
                              糖尿病有什么并发症<br/>\
                              糖尿病有什么症状"))#生成markdown格式的提示语言，可以看到utter_message除了引用domain的模板，也可以直接定义text内容
        return [] #返回一个空状态，，不对会话状态做任何处理。
```
同理，我们定义一个action_donknow，当NLU无法准确理解用户输入的时候，就调用这个action给用户提示。
```
class ActionDonKnow(Action):
    def name(self) -> Text:
        return "action_donknow"

    def run(self,
            dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]):
        dispatcher.utter_template("utter_donknow", tracker)
        # dispatcher.utter_template("utter_howcanhelp", tracker)
        dispatcher.utter_message(md("您可以这样向我提问: <br/>头痛怎么办<br/>\
                                      什么人容易头痛<br/>\
                                      头痛吃什么药<br/>\
                                      头痛能治吗<br/>\
                                      头痛属于什么科<br/>\
                                      头孢地尼分散片用途<br/>\
                                      如何防止头痛<br/>\
                                      头痛要治多久<br/>\
                                      糖尿病有什么并发症<br/>\
                                      糖尿病有什么症状"))
        return []
```
这个action的调用不在story里，而在我们的config文件里，当NLU对于用户输入意图判断的置信度无法达到阈值，就会fallback，然后执行对于的action：
```
  - name: "rasa.core.policies.fallback.FallbackPolicy"
    nlu_threshold: 0.4
    core_threshold: 0.3
    fallback_action_name: 'action_donknow'
```
* 查节点属性的查询（比如根据我们图数据库的schema，查治疗方法就属于disease节点的一个属性）
```
class ActionSearchTreat(Action):
    def name(self) -> Text:
        return "action_search_treat"
    def run(self,
            dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]):
        disease = tracker.get_slot("disease") #NLU提取到实体，都会存储在Tracker里，因此拿到它，直接从tracker里通过get_slot拿到
        pre_disease = tracker.get_slot("sure") #这是构建的一个辅助实体，方便生成button供用户选择
        print("pre_disease::::" + str(pre_disease))
        possible_diseases = retrieve_disease_name(disease)
        # if len(possible_diseases) == 1 or sure == "true":
        if disease == pre_disease or len(possible_diseases) == 1:
            a = graph.run("match (a:Disease{name: {disease}}) return a", disease=disease).data()[0]['a']
            if "intro" in a:
                intro = a['intro']
                template = "{0}的简介：{1}"
                retmsg = template.format(disease, intro)
            else:
                retmsg = disease + "暂无简介"
            dispatcher.utter_message(retmsg)
            if "treat" in a:
                treat = a['treat']
                template = "{0}的治疗方式有：{1}"
                retmsg = template.format(disease, "、".join(treat))
            else:
                retmsg = disease + "暂无常见治疗方式"
            dispatcher.utter_message(retmsg)
        elif len(possible_diseases) > 1:
            buttons = []
            for d in possible_diseases:
                buttons.append(make_button(d, '/search_treat{{"disease":"{0}", "sure":"{1}"}}'.format(d, d)))
            dispatcher.utter_button_message("请点击选择想查询的疾病，若没有想要的，请忽略此消息", buttons)
        else:
            dispatcher.utter_message("知识库中暂无与 {0} 疾病相关的记录".format(disease))
        return []
```
* 节点与节点之间的查询
```
class ActionSearchSymptom(Action):
    def name(self) -> Text:
        return "action_search_symptom"

    def run(self,
            dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]):
        disease = tracker.get_slot("disease")
        pre_disease = tracker.get_slot("sure")
        print("pre_disease::::" + str(pre_disease))
        possible_diseases = retrieve_disease_name(disease)
        if disease == pre_disease or len(possible_diseases) == 1:
            a = [x['s.name'] for x in graph.run("MATCH (p:Disease{name: {disease}})-[r:has_symptom]->\
                                                (s:Symptom) RETURN s.name", disease=disease).data()]
            template = "{0}的症状可能有：{1}"
            retmsg = template.format(disease, "、".join(a))
            dispatcher.utter_message(retmsg)
        elif len(possible_diseases) > 1:
            buttons = []
            for d in possible_diseases:
                buttons.append(make_button(d, '/search_symptom{{"disease":"{0}", "sure":"{1}"}}'.format(d, d)))
            dispatcher.utter_button_message("请点击选择想查询的疾病，若没有想要的，请忽略此消息", buttons)
        else:
            dispatcher.utter_message("知识库中暂无与 {0} 相关的症状记录".format(disease))

        return []
```
```json
请大家思考，为什么需要一个辅助变量“sure”？？？
```

#### 根据以上思路，实现以下查询
1. action_search_food
2. action_search_symptom
3. action_search_cause
4. action_search_treat
5. action_search_neopathy
6. action_search_drug
7. action_search_prevention
8. action_search_drug_func

## 2.训练模型
>rasa train

## 3.效果展示
* 启动action
>rasa run actions --port 5055 --actions actions --debug
![](img/rasa_run_actions.png)

* 开启一个会话shell
>rasa shell
![](img/rasa_shell.png)

* 输入你好问候语（intent：greed）
![](img/greet.png)

* 输入肺炎，通过模糊查询到所有类型的肺炎供用户选择
![](img/disease_select.png)

* 根据选择，得到该疾病的治疗建议
![](img/disease_input.png)

* 不需要额外提供疾病名字，继续提问基于上一轮聊天疾病的饮食注意什么（因为tracker里存储了）
![](img/diet_precautions.png)

* 再更换主体，询问药物一般用于哪些疾病
![](img/new_question.png)


#### 技术优化点：
* FormAction实战，基于医疗图谱，将动态设立槽位，2个action完成所有任务
* FAQ嵌入，增强问答能力
* 如何与前端协同部署？